In [3]:
import pymongo
from pymongo import MongoClient
import json
from pprint import pprint
from bson.json_util import loads
from bson.son import SON

In [4]:
# Use your own file path (e.g., C:\DSC3010\movieDetails.json)
file_path = 'C://DSC3010//movieDetails.json'
# To see the file structure
# 1) Open the file using a text editor
# 2) Copy-paste a single line to the "Text" tab of http://jsonviewer.stack.hu/
# 3) Open "Viewer" tab to see the structure

In [5]:
# Create a list
movies = list()

In [6]:
# Open the json file and add movies to the list
with open(file_path, encoding = "UTF-8") as f:
    for movie in f.readlines():
        movies.append(movie)

In [7]:
# The total number of movies
print(len(movies))

2295


### QUESTIONS (Please Write your code below and submit the ipynb file)
#### Q1: Create a "movies" collection in the "test" database
#### Q2: Insert all the 2295 movies to the "movies" collectoin.
#### Q3: How many genres are there? 
#### Q4: What are the top10 movies with the highest imdb rating?
#### Q5: What actor/actress has acted in the most number of movies?
#### Q6: What are the movies released between 2000 and 2010?
#### Q7: Find movies with genre "Animation" and "Action".
#### Q8: What are movies rated "G" or "PG"?
#### Q9: If you create indexes for the movies collection, which fields will you choose to create indexes on? and why?
#### Q10: Compared with a relational database, what are the advatanges of using MongoDB to host movie data?

## 1. create a "movies" collection in the "test" database


In [18]:
#1. create a "movies" collection in the "test" database
#create "test" database
myClient = pymongo.MongoClient('localhost', 27017)
db = myClient['test']

#create "movies" collection
mcol = db["movies"]

## 2. Insert all the 2295 movies to the "movies" collectoin.

In [19]:
#replace invalid characters with empty spaces

mod_movies = list()
for movie in movies:
    mod_movie = movie.replace("$", "")
    mod_movies.append(mod_movie)

In [20]:
#to check if there is any error in inserting a json file
count = 0
for movie in movies:
    
    try:
        movie = json.loads(movie)
        insert_result = mcol.insert_one(movie)
    except:
        count += 1

print(count)


2295


In [21]:
#create an index
mcol.create_index([('oid', pymongo.ASCENDING)], unique=True, sparse=True)
#['_id_', 'oid_1']
sorted(list(mcol.index_information()))

['_id_', 'oid_1']

In [22]:
mcol.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'test.movies'},
 'oid_1': {'v': 2,
  'unique': True,
  'key': [('oid', 1)],
  'ns': 'test.movies',
  'sparse': True}}

In [23]:
for movie in mod_movies:
    try:
        mdata = json.loads(movie)
        result = db.mcol.insert_one(mdata)
    except pymongo.errors.DuplicateKeyError:
        pass


In [24]:
mcol.estimated_document_count()

2299

## 3. how many genres are there?


In [25]:
#3. how many genres are there?
genre_doc = db.mcol.distinct("genres")
print("There are " + str(len(genre_doc)) + " genres")

There are 28 genres


## 4. what are the top 10 movies with the highest IMDB ratings

In [26]:
#4. what are the top 10 movies with the highest IMDB ratings
#db.mcol.find().sort({"rating" : -1}).limit(10)

mcol_sorted = db.mcol.find().sort("rating", pymongo.ASCENDING).limit(10)
for doc in mcol_sorted:
    pprint(doc)

{'_id': {'oid': '5b107bec1d2952d0da9046e0'},
 'actors': ['Claudia Cardinale',
            'Henry Fonda',
            'Jason Robards',
            'Charles Bronson'],
 'awards': {'nominations': 5.0, 'text': '4 wins & 5 nominations.', 'wins': 4.0},
 'countries': ['Italy', 'USA', 'Spain'],
 'director': 'Sergio Leone',
 'genres': ['Western'],
 'imdb': {'id': 'tt0064116', 'rating': 8.6, 'votes': 201283.0},
 'metacritic': 80.0,
 'plot': 'Epic story of a mysterious stranger with a harmonica who joins '
         'forces with a notorious desperado to protect a beautiful widow from '
         'a ruthless assassin working for the railroad.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMTEyODQzNDkzNjVeQTJeQWpwZ15BbWU4MDgyODk1NDEx._V1_SX300.jpg',
 'rated': 'PG-13',
 'runtime': 175.0,
 'title': 'Once Upon a Time in the West',
 'tomato': {'consensus': 'A landmark Sergio Leone spaghetti western '
                         'masterpiece featuring a classic Morricone score.',
            'fresh': 53

## 5. what actor/actress has acted in the most number of movies


In [27]:
#5. what actor/actress has acted in the most number of movies
#db.mcol.aggregate(
#    {"$group" : {"_id" : "actors", "count" : {"$sum" : 1}}}
#)


    
dactors =  db.mcol.distinct("actors")
print(len(dactors))

actors = db.mcol.find({},{"actors"})

actors_occ = db.mcol.aggregate([
    {"$unwind":"$actors"},
    {"$group":{"_id":"$actors", "sum":{"$sum":1}}},
])

actors_sort = sorted(actors_occ, key=lambda x: x['sum'], reverse = True)

#sort the number of appearance by actors/actresses
actors_sort

#Louis C.K, Tom Hanks, Natalie Portman have acted in 8 movies and
# hence they are the ones with the most number of movies acted in.





6401


[{'_id': 'Louis C.K.', 'sum': 8},
 {'_id': 'Tom Hanks', 'sum': 8},
 {'_id': 'Natalie Portman', 'sum': 8},
 {'_id': 'Shah Rukh Khan', 'sum': 7},
 {'_id': 'B.B. King', 'sum': 7},
 {'_id': 'Scarlett Johansson', 'sum': 7},
 {'_id': 'Ewan McGregor', 'sum': 7},
 {'_id': 'Cliff Williams', 'sum': 6},
 {'_id': 'Brian Johnson', 'sum': 6},
 {'_id': 'Angus Young', 'sum': 6},
 {'_id': 'Ben Stiller', 'sum': 6},
 {'_id': 'Patricia Arquette', 'sum': 6},
 {'_id': 'O.J. Simpson', 'sum': 6},
 {'_id': 'Adam Sandler', 'sum': 6},
 {'_id': 'Will Smith', 'sum': 6},
 {'_id': 'T.T. Boy', 'sum': 5},
 {'_id': 'Pink', 'sum': 5},
 {'_id': 'D.L. Hughley', 'sum': 5},
 {'_id': 'Arve Opsahl', 'sum': 5},
 {'_id': 'Amitabh Bachchan', 'sum': 5},
 {'_id': 'Steve Buscemi', 'sum': 5},
 {'_id': 'Aamir Khan', 'sum': 5},
 {'_id': 'Kevin Spacey', 'sum': 5},
 {'_id': 'Angelina Jolie', 'sum': 5},
 {'_id': 'Carla Gugino', 'sum': 5},
 {'_id': 'Dustin Hoffman', 'sum': 5},
 {'_id': 'Gérard Depardieu', 'sum': 5},
 {'_id': 'Akshay Kumar

## 6. what are the movies released between 2000 and 2010
 

In [28]:
#6 what are the movies released between 2000 and 2010
mcol_released = db.mcol.find({"year":{"$gte":2000, "$lte":2010}})
for doc in mcol_released:
    pprint(doc)

{'_id': {'oid': '5b107bec1d2952d0da9046ee'},
 'actors': ['Ewan McGregor',
            'Natalie Portman',
            'Hayden Christensen',
            'Ian McDiarmid'],
 'awards': {'nominations': 49.0,
            'text': 'Nominated for 1 Oscar. Another 21 wins & 49 nominations.',
            'wins': 21.0},
 'countries': ['USA'],
 'director': 'George Lucas',
 'genres': ['Action', 'Adventure', 'Fantasy'],
 'imdb': {'id': 'tt0121766', 'rating': 7.6, 'votes': 483613.0},
 'metacritic': 68.0,
 'plot': 'Three years after the onset of the Clone Wars; the noble Jedi '
         'Knights are spread out across the galaxy leading a massive clone '
         'army in the war against the Separatists. After Chancellor ...',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BNTc4MTc3NTQ5OF5BMl5BanBnXkFtZTcwOTg0NjI4NA@@._V1_SX300.jpg',
 'rated': 'PG-13',
 'runtime': 140.0,
 'title': 'Star Wars: Episode III - Revenge of the Sith',
 'tomato': {'consensus': 'With Episode III: Revenge of the Sith, George Luc

 'tomato': {'consensus': 'A kinetic, violent and surprisingly worthy remake of '
                         "George Romero's horror classic that pays homage to "
                         'the original while working on its own terms.',
            'fresh': 135.0,
            'image': 'certified',
            'meter': 75.0,
            'rating': 6.7,
            'reviews': 180.0,
            'userMeter': 77.0,
            'userRating': 3.4,
            'userReviews': 402786.0},
 'type': 'movie',
 'writers': ['George A. Romero', 'James Gunn'],
 'year': 2004.0}
{'_id': {'oid': '5b107bec1d2952d0da904725'},
 'actors': ['Philip Seymour Hoffman',
            'Ethan Hawke',
            'Albert Finney',
            'Marisa Tomei'],
 'awards': {'nominations': 25.0,
            'text': '17 wins & 25 nominations.',
            'wins': 17.0},
 'countries': ['USA'],
 'director': 'Sidney Lumet',
 'genres': ['Crime', 'Drama', 'Thriller'],
 'imdb': {'id': 'tt0292963', 'rating': 7.3, 'votes': 79728.0},
 'm

            'Jerry Stiller'],
 'awards': {'nominations': 0.0, 'text': '1 win.', 'wins': 1.0},
 'countries': ['USA'],
 'director': 'Bobby Farrelly, Peter Farrelly',
 'genres': ['Comedy', 'Romance'],
 'imdb': {'id': 'tt0408839', 'rating': 5.8, 'votes': 68873.0},
 'metacritic': 46.0,
 'plot': "A newly wed man who believes he's just gotten hitched to the perfect "
         'woman encounters another lady on his honeymoon.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMjExNTM2NTE2NF5BMl5BanBnXkFtZTcwOTI5MDUzMw@@._V1_SX300.jpg',
 'rated': 'R',
 'runtime': 116.0,
 'title': 'The Heartbreak Kid',
 'tomato': {'consensus': 'Despite some amiable performances, The Heartbreak '
                         'Kid is neither as daring nor as funny as the '
                         "Farrelly Brothers' earlier films.",
            'fresh': 45.0,
            'image': 'rotten',
            'meter': 29.0,
            'rating': 4.6,
            'reviews': 156.0,
            'userMeter': 37.0,
            'u

 'actors': ['Kôichi Yamadera',
            'Unshô Ishizuka',
            'Megumi Hayashibara',
            'Aoi Tada'],
 'awards': {'nominations': 1.0, 'text': '1 nomination.', 'wins': 0.0},
 'countries': ['Japan', 'USA'],
 'director': 'Shinichirô Watanabe, Hiroyuki Okiura',
 'genres': ['Animation', 'Action', 'Crime'],
 'imdb': {'id': 'tt0275277', 'rating': 7.9, 'votes': 29239.0},
 'metacritic': 61.0,
 'plot': 'A terrorist explosion releases a deadly virus on the masses, and '
         "it's up the bounty-hunting Bebop crew to catch the cold-blooded "
         'culprit.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMTA1ODIxMDYyMzFeQTJeQWpwZ15BbWU2MDUwOTQ5Ng@@._V1_SX300.jpg',
 'rated': 'R',
 'runtime': 115.0,
 'title': 'Cowboy Bebop: The Movie',
 'tomato': {'consensus': "This may be strictly for anime junkies, but they'll "
                         "find much to like about Cowboy Bebop's casual "
                         'violence and cool dialogue.',
            'fresh': 45.0,
  

 'poster': 'http://ia.media-imdb.com/images/M/MV5BMTMxMTIyNzMxMV5BMl5BanBnXkFtZTcwOTc4OTI3Mg@@._V1_SX300.jpg',
 'rated': 'R',
 'runtime': 138.0,
 'title': 'Shutter Island',
 'tomato': {'consensus': "It may not rank with Scorsese's best work, but "
                         "Shutter Island's gleefully unapologetic genre "
                         'thrills represent the director at his most '
                         'unrestrained.',
            'fresh': 165.0,
            'image': 'fresh',
            'meter': 68.0,
            'rating': 6.6,
            'reviews': 241.0,
            'userMeter': 76.0,
            'userRating': 3.3,
            'userReviews': 2370815.0},
 'type': 'movie',
 'writers': ['Laeta Kalogridis', 'Dennis Lehane'],
 'year': 2010.0}
{'_id': {'oid': '5b107bec1d2952d0da90479d'},
 'actors': ['Ewan McGregor',
            'Scarlett Johansson',
            'Djimon Hounsou',
            'Sean Bean'],
 'awards': {'nominations': 6.0, 'text': '1 win & 6 nominations.', 'wins'

...

 'type': 'movie',
 'writers': [],
 'year': 2002.0}
{'_id': {'oid': '5b107bec1d2952d0da904e97'},
 'actors': ['C.J. Knight', 'Rod Barry', 'Jude Collin', 'Chad Hunt'],
 'awards': {'nominations': 0.0, 'text': '', 'wins': 0.0},
 'countries': ['USA'],
 'director': 'Michael Clift',
 'genres': ['Adult'],
 'imdb': {'id': 'tt1155710', 'rating': None, 'votes': None},
 'plot': None,
 'poster': None,
 'rated': 'X',
 'runtime': 90.0,
 'title': 'Pack Attack 3: C.J. Knight',
 'type': 'movie',
 'writers': [],
 'year': 2007.0}
{'_id': {'oid': '5b107bec1d2952d0da904ea2'},
 'actors': [],
 'awards': {'nominations': 0.0, 'text': '', 'wins': 0.0},
 'countries': ['USA'],
 'director': 'Matthew Fults',
 'genres': ['Documentary'],
 'imdb': {'id': 'tt1786731', 'rating': None, 'votes': None},
 'plot': 'It all started with the passion of a few hearty New Englanders, the '
         'ingenuity of a socialite, and the commitment of a community. In its '
         'eight decades of hockey, the Colorado College Tigers ..

 'plot': None,
 'poster': None,
 'rated': None,
 'runtime': 3.0,
 'title': 'Ez-Chewie',
 'type': 'movie',
 'writers': ['Cameron McCasland'],
 'year': 2009.0}
{'_id': {'oid': '5b107bec1d2952d0da904eef'},
 'actors': [],
 'awards': {'nominations': 0.0, 'text': '', 'wins': 0.0},
 'countries': ['Hungary'],
 'director': 'Ágnes Sós',
 'genres': ['Documentary'],
 'imdb': {'id': 'tt2211189', 'rating': None, 'votes': None},
 'plot': None,
 'poster': None,
 'rated': None,
 'runtime': 74.0,
 'title': 'Örült szerelem ez',
 'type': 'movie',
 'writers': [],
 'year': 2008.0}
{'_id': {'oid': '5b107bec1d2952d0da904ef0'},
 'actors': ['Carolyn Morse', 'Erik Nicolaisen', 'Isabella Steketee'],
 'awards': {'nominations': 0.0, 'text': '', 'wins': 0.0},
 'countries': ['USA'],
 'director': 'Alex Warth',
 'genres': ['Short', 'Comedy'],
 'imdb': {'id': 'tt1858744', 'rating': None, 'votes': None},
 'plot': "Getting baked with Mom! A high brow parody of an 80's Easy Bake oven "
         'commercial.',
 'poster': No

 'poster': 'http://ia.media-imdb.com/images/M/MV5BMTU0NjA0ODk1Ml5BMl5BanBnXkFtZTcwNTU4OTAwMg@@._V1_SX300.jpg',
 'rated': None,
 'runtime': 61.0,
 'title': 'Louis C.K.: Chewed Up',
 'type': 'movie',
 'writers': ['Louis C.K.'],
 'year': 2008.0}
{'_id': {'oid': '5b107bec1d2952d0da904f2e'},
 'actors': ['Louis C.K.'],
 'awards': {'nominations': 0.0, 'text': '', 'wins': 0.0},
 'countries': ['USA'],
 'director': 'Louis C.K.',
 'genres': ['Documentary', 'Comedy'],
 'imdb': {'id': 'tt1421373', 'rating': 8.6, 'votes': 5444.0},
 'plot': "In this unique and dynamic live concert experience, Louis C.K.'s "
         'exploration of life after 40 destroys politically correct images of '
         'modern life with thoughts we have all had...but would rarely admit '
         'to.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMTY1MzQ1MTM5OV5BMl5BanBnXkFtZTcwMTg0MTQzNA@@._V1_SX300.jpg',
 'rated': None,
 'runtime': 82.0,
 'title': 'Louis C.K.: Hilarious',
 'type': 'movie',
 'writers': ['Louis C.K.'],

 'poster': 'http://ia.media-imdb.com/images/M/MV5BMTc1MDgwNDE4MF5BMl5BanBnXkFtZTcwMTQzMzc0MQ@@._V1_SX300.jpg',
 'rated': 'R',
 'runtime': 120.0,
 'title': "Paris, je t'aime",
 'tomato': {'consensus': "Paris Je T'aime is uneven, but there are more than "
                         'enough delightful moments in this omnibus tribute to '
                         'the City of Lights to tip the scale in its favor.',
            'fresh': 97.0,
            'image': 'certified',
            'meter': 87.0,
            'rating': 7.1,
            'reviews': 112.0,
            'userMeter': 84.0,
            'userRating': 3.8,
            'userReviews': 192094.0},
 'type': 'movie',
 'writers': ['Tristan Carné',
             'Emmanuel Benbihy',
             'Emmanuel Benbihy',
             'Bruno Podalydès',
             'Paul Mayeda Berges',
             'Gurinder Chadha',
             'Gus Van Sant',
             'Joel Coen',
             'Ethan Coen',
             'Walter Salles',
             'Dan

 'year': 2002.0}
{'_id': {'oid': '5b107bec1d2952d0da904fad'},
 'actors': [],
 'awards': {'nominations': 0.0, 'text': '', 'wins': 0.0},
 'countries': ['Germany'],
 'director': 'Sandra Prechtel, François Rossier',
 'genres': ['Documentary'],
 'imdb': {'id': 'tt0969726', 'rating': None, 'votes': None},
 'plot': None,
 'poster': None,
 'rated': None,
 'runtime': None,
 'title': 'ND - Deutsches Neuland',
 'type': 'movie',
 'writers': ['Sandra Prechtel', 'François Rossier'],
 'year': 2004.0}
{'_id': {'oid': '5b107bec1d2952d0da904faf'},
 'actors': ['Hillary Rodham Clinton',
            'Bill Cosby',
            'John Hope Franklin',
            'Jim Hunt'],
 'awards': {'nominations': 0.0, 'text': '', 'wins': 0.0},
 'countries': ['USA'],
 'director': 'Cash Michaels',
 'genres': ['Documentary'],
 'imdb': {'id': 'tt1615051', 'rating': 2.3, 'votes': 6.0},
 'plot': 'Never before has a film fully examined the historic election of Sen. '
         'Barack Obama to the presidency of the United States 

## 7. Find mvies with genre "Animation" and "Action"


In [29]:
#7. Find mvies with genre "Animation" and "Action"

for doc in db.mcol.find({"genres":{"$all":["Animation", "Action"]}}):
    pprint(doc)


{'_id': {'oid': '5b107bec1d2952d0da904708'},
 'actors': ['Jamie Bell', 'Andy Serkis', 'Daniel Craig', 'Nick Frost'],
 'awards': {'nominations': 58.0,
            'text': 'Nominated for 1 Oscar. Another 22 wins & 58 nominations.',
            'wins': 22.0},
 'countries': ['USA', 'New Zealand'],
 'director': 'Steven Spielberg',
 'genres': ['Animation', 'Action', 'Adventure'],
 'imdb': {'id': 'tt0983193', 'rating': 7.4, 'votes': 169875.0},
 'metacritic': 68.0,
 'plot': 'Intrepid reporter Tintin and Captain Haddock set off on a treasure '
         "hunt for a sunken ship commanded by Haddock's ancestor.",
 'poster': 'http://ia.media-imdb.com/images/M/MV5BNDE5MDExNTQ1OF5BMl5BanBnXkFtZTcwMDIxMTM5Ng@@._V1_SX300.jpg',
 'rated': 'PG',
 'runtime': 107.0,
 'title': 'The Adventures of Tintin',
 'tomato': {'consensus': 'Drawing deep from the classic Raiders of the Lost '
                         'Ark playbook, Steven Spielberg has crafted another '
                         'spirited, thrilling adve

## 8. what are movies rated "G" or "PG"?
 

In [30]:
#8. what are movies rated "G" or "PG"?
mcol_rated = db.mcol.find({"$or":[{"rated":"G"}, {"rated":"PG"}]})

for doc in mcol_rated:
    pprint(doc)

{'_id': {'oid': '5b107bec1d2952d0da9046e5'},
 'actors': ['Phillip Glasser', 'James Stewart', 'Erica Yohn', 'Cathy Cavadini'],
 'awards': {'nominations': 0.0, 'text': '', 'wins': 0.0},
 'countries': ['USA'],
 'director': 'Phil Nibbelink, Simon Wells',
 'genres': ['Animation', 'Adventure', 'Family'],
 'imdb': {'id': 'tt0101329', 'rating': 6.4, 'votes': 16013.0},
 'plot': 'A family of Emigre mice decide to move out to the west, unaware that '
         'they are falling into a trap perpetrated by a smooth talking cat.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMTYzODMxNTUyNF5BMl5BanBnXkFtZTcwMTAwMzQyMQ@@._V1_SX300.jpg',
 'rated': 'G',
 'runtime': 75.0,
 'title': 'An American Tail: Fievel Goes West',
 'type': 'movie',
 'writers': ['Flint Dille', 'Charles Swenson', 'David Kirschner'],
 'year': 1991.0}
{'_id': {'oid': '5b107bec1d2952d0da9046ea'},
 'actors': ['Mark Hamill', 'Harrison Ford', 'Carrie Fisher', 'Peter Cushing'],
 'awards': {'nominations': 27.0,
            'text': 'Won 6 

            'meter': 96.0,
            'rating': 9.2,
            'reviews': 70.0,
            'userMeter': 89.0,
            'userRating': 3.8,
            'userReviews': 294447.0},
 'type': 'movie',
 'writers': ['Stanley Kubrick', 'Arthur C. Clarke'],
 'year': 1968.0}
{'_id': {'oid': '5b107bec1d2952d0da904700'},
 'actors': ['Jonah Bobo', 'Josh Hutcherson', 'Dax Shepard', 'Kristen Stewart'],
 'awards': {'nominations': 3.0, 'text': '2 wins & 3 nominations.', 'wins': 2.0},
 'countries': ['USA'],
 'director': 'Jon Favreau',
 'genres': ['Action', 'Adventure', 'Comedy'],
 'imdb': {'id': 'tt0406375', 'rating': 6.1, 'votes': 62971.0},
 'metacritic': 67.0,
 'plot': 'Two young brothers are drawn into an intergalactic adventure when '
         'their house is hurled through the depths of space by the magical '
         'board game they are playing.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMjEyOTQ2ODg4OV5BMl5BanBnXkFtZTcwMjk2ODUzMQ@@._V1_SX300.jpg',
 'rated': 'PG',
 'runtime': 101.0,


 'runtime': 103.0,
 'title': 'The Adventures of Buckaroo Banzai Across the 8th Dimension',
 'type': 'movie',
 'writers': ['Earl Mac Rauch'],
 'year': 1984.0}
{'_id': {'oid': '5b107bec1d2952d0da904710'},
 'actors': ['Rene Russo', 'Jason Alexander', 'Piper Perabo', 'Randy Quaid'],
 'awards': {'nominations': 4.0, 'text': '2 wins & 4 nominations.', 'wins': 2.0},
 'countries': ['USA', 'Germany'],
 'director': 'Des McAnuff',
 'genres': ['Animation', 'Adventure', 'Comedy'],
 'imdb': {'id': 'tt0131704', 'rating': 4.1, 'votes': 16248.0},
 'metacritic': 36.0,
 'plot': 'When enemies Boris, Natasha and Fearless leader escape into the real '
         'world with a nefarious scheme, Rocky & Bullwinkle do the same and '
         'team up with a young F.B.I. agent to stop the trio.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMzI0Mzk2NzcwNV5BMl5BanBnXkFtZTcwNjYyNTkxMQ@@._V1_SX300.jpg',
 'rated': 'PG',
 'runtime': 92.0,
 'title': 'The Adventures of Rocky & Bullwinkle',
 'tomato': {'consensus': '

            'wins': 5.0},
 'countries': ['USA', 'China'],
 'director': 'Harald Zwart',
 'genres': ['Action', 'Drama', 'Family'],
 'imdb': {'id': 'tt1155076', 'rating': 6.2, 'votes': 113299.0},
 'metacritic': 61.0,
 'plot': 'Work causes a single mother to move to China with her young son; in '
         'his new home, the boy embraces kung fu, taught to him by a master.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMTQ0ODg3ODEyMF5BMl5BanBnXkFtZTcwNjI1MTgxMw@@._V1_SX300.jpg',
 'rated': 'PG',
 'runtime': 140.0,
 'title': 'The Karate Kid',
 'tomato': {'consensus': 'It may not be as powerful as the 1984 edition, but '
                         'the 2010 Karate Kid delivers a surprisingly '
                         'satisfying update on the original.',
            'fresh': 133.0,
            'image': 'fresh',
            'meter': 66.0,
            'rating': 6.2,
            'reviews': 203.0,
            'userMeter': 67.0,
            'userRating': 3.5,
            'userReviews': 281741.0

            'fresh': 38.0,
            'image': 'certified',
            'meter': 88.0,
            'rating': 7.8,
            'reviews': 43.0,
            'userMeter': 79.0,
            'userRating': 3.0,
            'userReviews': 112623.0},
 'type': 'movie',
 'writers': ['Phil Hartman', 'Paul Reubens', 'Michael Varhol'],
 'year': 1985.0}
{'_id': {'oid': '5b107bec1d2952d0da904778'},
 'actors': ['Gene Hackman', 'Ernest Borgnine', 'Red Buttons', 'Carol Lynley'],
 'awards': {'nominations': 13.0,
            'text': 'Won 1 Oscar. Another 4 wins & 13 nominations.',
            'wins': 4.0},
 'countries': ['USA'],
 'director': 'Ronald Neame, Irwin Allen',
 'genres': ['Action', 'Adventure', 'Drama'],
 'imdb': {'id': 'tt0069113', 'rating': 7.1, 'votes': 31707.0},
 'plot': 'A group of passengers struggle to survive and escape when their '
         'ocean liner completely capsizes at sea.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMTczNDU2MzIzNl5BMl5BanBnXkFtZTYwNjg0MDY5._V1_SX300.jpg

...

            'wins': 44.0},
 'countries': ['India'],
 'director': 'Ashutosh Gowariker',
 'genres': ['Adventure', 'Drama', 'Musical'],
 'imdb': {'id': 'tt0169102', 'rating': 8.2, 'votes': 64633.0},
 'metacritic': 84.0,
 'plot': 'The people of a small village in Victorian India stake their future '
         'on a game of cricket against their ruthless British rulers.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMTIwODMwMzA5Ml5BMl5BanBnXkFtZTcwMTQxNTEyMQ@@._V1_SX300.jpg',
 'rated': 'PG',
 'runtime': 224.0,
 'title': 'Lagaan: Once Upon a Time in India',
 'tomato': {'consensus': 'Lagaan is lavish, rousing entertainment in the '
                         'old-fashioned tradition of Hollywood musicals.',
            'fresh': 56.0,
            'image': 'certified',
            'meter': 95.0,
            'rating': 7.9,
            'reviews': 59.0,
            'userMeter': 93.0,
            'userRating': 4.2,
            'userReviews': 19466.0},
 'type': 'movie',
 'writers': ['Ashutosh Gowa

            'userMeter': 86.0,
            'userRating': 3.5,
            'userReviews': 995597.0},
 'type': 'movie',
 'writers': ['John Lasseter',
             'Pete Docter',
             'Ash Brannon',
             'Andrew Stanton',
             'Andrew Stanton',
             'Rita Hsiao',
             'Doug Chamberlin',
             'Chris Webb'],
 'year': 1999.0}
{'_id': {'oid': '5b107bec1d2952d0da904828'},
 'actors': ['Melinda Dillon',
            'Darren McGavin',
            'Peter Billingsley',
            'Scott Schwartz'],
 'awards': {'nominations': 11.0,
            'text': '3 wins & 11 nominations.',
            'wins': 3.0},
 'countries': ['USA', 'Canada'],
 'director': 'Bob Clark',
 'genres': ['Comedy', 'Family'],
 'imdb': {'id': 'tt0085334', 'rating': 8.1, 'votes': 100651.0},
 'metacritic': 77.0,
 'plot': 'Ralphie has to convince his parents, his teacher, and Santa that a '
         'Red Ryder B.B. gun really is the perfect gift for the 1940s.',
 'poster': 'http://ia.med

 'type': 'movie',
 'writers': ['L. Frank Baum', 'Gill Dennis', 'Walter Murch'],
 'year': 1985.0}
{'_id': {'oid': '5b107bec1d2952d0da904abe'},
 'actors': ['James Franco', 'Mila Kunis', 'Rachel Weisz', 'Michelle Williams'],
 'awards': {'nominations': 31.0,
            'text': '5 wins & 31 nominations.',
            'wins': 5.0},
 'countries': ['USA'],
 'director': 'Sam Raimi',
 'genres': ['Adventure', 'Family', 'Fantasy'],
 'imdb': {'id': 'tt1623205', 'rating': 6.4, 'votes': 169825.0},
 'metacritic': 44.0,
 'plot': 'A small-time magician is swept away to an enchanted land and is '
         'forced into a power struggle between three witches.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMjMyMzQ1ODM1MF5BMl5BanBnXkFtZTcwMjE2MTQxOQ@@._V1_SX300.jpg',
 'rated': 'PG',
 'runtime': 130.0,
 'title': 'Oz the Great and Powerful',
 'tomato': {'consensus': 'It suffers from some tonal inconsistency and a '
                         'deflated sense of wonder, but Oz the Great and '
               

 'runtime': 98.0,
 'title': 'Oh, God!',
 'type': 'movie',
 'writers': ['Larry Gelbart', 'Avery Corman'],
 'year': 1977.0}
{'_id': {'oid': '5b107bec1d2952d0da904c83'},
 'actors': ['Gregory Abbey', 'Sebastian Arcelus', 'Ben Baron', 'Amy Birnbaum'],
 'awards': {'nominations': 0.0, 'text': '', 'wins': 0.0},
 'countries': ['Japan', 'USA'],
 'director': 'Hatsuki Tsuji',
 'genres': ['Animation', 'Action', 'Adventure'],
 'imdb': {'id': 'tt0403703', 'rating': 4.4, 'votes': 6077.0},
 'metacritic': 15.0,
 'plot': 'Underneath the sands of Egypt, Anubis, an ancient evil spirit, has '
         "awakened. It's up to Yugi, who defeated Anubis centuries ago, to use "
         'his skill and determination to rid the world of evil once again.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BNTQ4NzM0NTAyMF5BMl5BanBnXkFtZTYwOTc2MTg3._V1_SX300.jpg',
 'rated': 'PG',
 'runtime': 89.0,
 'title': 'Yu-Gi-Oh!: The Movie',
 'tomato': {'consensus': "Don't watch the TV show or play the card game? Then "
         

 'director': 'Jean-Pierre Jeunet',
 'genres': ['Action', 'Adventure', 'Drama'],
 'imdb': {'id': 'tt1981107', 'rating': 7.1, 'votes': 10029.0},
 'metacritic': 53.0,
 'plot': "A ten-year-old cartographer secretly leaves his family's ranch in "
         'Montana where he lives with his cowboy father and scientist mother '
         'and travels across the country aboard a freight train to receive an '
         'award at the Smithsonian Institute.',
 'poster': 'http://ia.media-imdb.com/images/M/MV5BMTk3MDU1MTc5Ml5BMl5BanBnXkFtZTgwNzk2NDY2NjE@._V1_SX300.jpg',
 'rated': 'PG',
 'runtime': 105.0,
 'title': 'The Young and Prodigious T.S. Spivet',
 'tomato': {'consensus': None,
            'fresh': 32.0,
            'image': 'fresh',
            'meter': 78.0,
            'rating': 6.0,
            'reviews': 41.0,
            'userMeter': 70.0,
            'userRating': 3.6,
            'userReviews': 1447.0},
 'type': 'movie',
 'writers': ['Jean-Pierre Jeunet',
             'Guillaume Laurant',

## 9. if you create indexes for the moveis collection, which fields will you choose to create indexes on? and why?

I would create indexes on _id and oid which are unique identifiers of each movie details. 
With indexes on _id and oid, it will be faster to locate data without searching every single row in a database.

## 10. Compared with a relational database, what are the advantages of using MongoDB to host movie data?

Movie data is lively or occassionally updated which means that there will be consistent inflow of data into the database. With a large amount of data being flown, the structure of database might need to be modified and under such environment, MongoDB is more flexible as compared to RDBMS